In [1]:
import numpy as np  # linear algebra
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv)
import os
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
from PIL import Image
#from imutils import paths
import random
import pickle



from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *

import keras
from keras.utils import np_utils
from keras.regularizers import *
from keras.initializers import glorot_uniform

import keras.backend as K
K.clear_session()

from keras.models import *
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.callbacks import *

In [4]:

benign=pd.read_csv('1.benign.csv')
g_c=pd.read_csv('1.gafgyt.combo.csv')
g_j=pd.read_csv('1.gafgyt.junk.csv')
g_s=pd.read_csv('1.gafgyt.scan.csv')
g_t=pd.read_csv('1.gafgyt.tcp.csv')
g_u=pd.read_csv('1.gafgyt.udp.csv')
m_a=pd.read_csv('1.mirai.ack.csv')
m_sc=pd.read_csv('1.mirai.scan.csv')
m_sy=pd.read_csv('1.mirai.syn.csv')
m_u=pd.read_csv('1.mirai.udp.csv')
m_u_p=pd.read_csv('1.mirai.udpplain.csv')

In [5]:
benign['type']='benign'
m_u['type']='mirai_udp'
g_c['type']='gafgyt_combo'
g_j['type']='gafgyt_junk'
g_s['type']='gafgyt_scan'
g_t['type']='gafgyt_tcp'
g_u['type']='gafgyt_udp'
m_a['type']='mirai_ack'
m_sc['type']='mirai_scan'
m_sy['type']='mirai_syn'
m_u_p['type']='mirai_udpplain'

In [6]:
data=pd.concat([benign,m_u,g_c,g_j,g_s,g_t,g_u,m_a,m_sc,m_sy,m_u_p],
               axis=0, sort=False, ignore_index=True)

In [7]:
data.groupby('type')['type'].count()

type
benign             49548
gafgyt_combo       59718
gafgyt_junk        29068
gafgyt_scan        29849
gafgyt_tcp         92141
gafgyt_udp        105874
mirai_ack         102195
mirai_scan        107685
mirai_syn         122573
mirai_udp         237665
mirai_udpplain     81982
Name: type, dtype: int64

In [8]:
sampler=np.random.permutation(len(data))
data=data.take(sampler)
data

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
665938,170.228987,290.369736,6.347957e+04,256.044816,314.863526,63993.934128,672.700840,360.628375,61735.540256,6382.152335,...,0.0,0.0,1.000000,566.0,0.000000e+00,566.000000,0.000000e+00,0.0,0.0,mirai_ack
337611,156.604250,74.059858,1.433005e+00,257.622474,74.059165,1.416459,779.698709,74.050001,1.199162,7359.231728,...,0.0,0.0,1.677271,74.0,9.536743e-07,95.268043,9.094947e-13,0.0,0.0,gafgyt_combo
605811,127.427797,356.147117,6.214733e+04,202.890003,391.068930,57914.242124,616.461552,402.176822,56056.490714,5508.984457,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,mirai_ack
651824,148.603764,269.438290,6.211138e+04,225.081425,322.538656,63918.014035,630.411330,373.741484,60319.466814,6264.575578,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,mirai_ack
959350,145.195746,188.680328,4.700901e+04,209.221232,236.191235,55993.075955,533.503812,298.446571,60930.581099,5018.111795,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,mirai_udpplain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518146,1.000000,60.000000,0.000000e+00,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,gafgyt_udp
714831,28.071069,60.000000,5.911716e-11,46.008651,60.000000,0.000003,182.177501,60.000989,0.047141,2223.697907,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,mirai_scan
873640,111.302373,72.464193,1.914284e+01,199.534499,71.318127,30.364821,655.742442,69.998127,40.284629,6661.627617,...,0.0,0.0,1.000000,74.0,0.000000e+00,74.000000,0.000000e+00,0.0,0.0,mirai_syn
432884,1.000000,60.000000,0.000000e+00,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,gafgyt_tcp


In [9]:
labels_full=pd.get_dummies(data['type'], prefix='type')
labels_full.head()

,type_benign,type_gafgyt_combo,type_gafgyt_junk,type_gafgyt_scan,type_gafgyt_tcp,type_gafgyt_udp,type_mirai_ack,type_mirai_scan,type_mirai_syn,type_mirai_udp,type_mirai_udpplain
665938,0,0,0,0,0,0,1,0,0,0,0
337611,0,1,0,0,0,0,0,0,0,0,0
605811,0,0,0,0,0,0,1,0,0,0,0
651824,0,0,0,0,0,0,1,0,0,0,0
959350,0,0,0,0,0,0,0,0,0,0,1


In [10]:
#drop labels from training dataset
data=data.drop(columns='type')
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
665938,170.228987,290.369736,63479.565739,256.044816,314.863526,63993.934128,672.700840,360.628375,61735.540256,6382.152335,...,0.0,0.0,0.0,1.000000,566.0,0.000000e+00,566.000000,0.000000e+00,0.0,0.0
337611,156.604250,74.059858,1.433005,257.622474,74.059165,1.416459,779.698709,74.050001,1.199162,7359.231728,...,0.0,0.0,0.0,1.677271,74.0,9.536743e-07,95.268043,9.094947e-13,0.0,0.0
605811,127.427797,356.147117,62147.326311,202.890003,391.068930,57914.242124,616.461552,402.176822,56056.490714,5508.984457,...,0.0,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0
651824,148.603764,269.438290,62111.377397,225.081425,322.538656,63918.014035,630.411330,373.741484,60319.466814,6264.575578,...,0.0,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0
959350,145.195746,188.680328,47009.006733,209.221232,236.191235,55993.075955,533.503812,298.446571,60930.581099,5018.111795,...,0.0,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0


In [11]:
#standardize numerical columns
def standardize(df,col):
    df[col]= (df[col]-df[col].mean())/df[col].std()

data_st=data.copy()
for i in (data_st.iloc[:,:-1].columns):
    standardize (data_st,i)

data_st.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
665938,1.182302,0.598518,1.738917,1.046093,0.775571,1.568911,0.817600,1.087345,1.371660,0.806911,...,-0.011451,-0.009765,-0.011037,-0.205909,1.710807,-0.015320,1.704178,-0.011606,-0.010185,0.0
337611,0.968751,-0.727278,-0.782500,1.061699,-0.763079,-0.821976,1.182296,-0.791004,-0.842398,1.155222,...,-0.011451,-0.009765,-0.011037,-0.205721,-0.560508,-0.015319,-0.476363,-0.011606,-0.010185,0.0
605811,0.511447,1.001677,1.685999,0.520266,1.262495,1.341761,0.625912,1.359670,1.167985,0.495642,...,-0.011451,-0.009765,-0.011037,-0.205909,-0.625139,-0.015320,-0.639733,-0.011606,-0.010185,0.0
651824,0.843354,0.470226,1.684571,0.739792,0.824612,1.566074,0.673459,1.173293,1.320873,0.764997,...,-0.011451,-0.009765,-0.011037,-0.205909,-0.625139,-0.015320,-0.639733,-0.011606,-0.010185,0.0
959350,0.789937,-0.024752,1.084689,0.582897,0.272885,1.269983,0.343155,0.679781,1.342791,0.320655,...,-0.011451,-0.009765,-0.011037,-0.205909,-0.625139,-0.015320,-0.639733,-0.011606,-0.010185,0.0


In [12]:
#training data for the neural net
train_data_st=data_st.values
train_data_st

array([[ 1.18230226e+00,  5.98517501e-01,  1.73891689e+00, ...,
        -1.16061272e-02, -1.01853314e-02,  0.00000000e+00],
       [ 9.68751396e-01, -7.27277844e-01, -7.82500186e-01, ...,
        -1.16061272e-02, -1.01853314e-02,  0.00000000e+00],
       [ 5.11446690e-01,  1.00167683e+00,  1.68599896e+00, ...,
        -1.16061272e-02, -1.01853314e-02,  0.00000000e+00],
       ...,
       [ 2.58700671e-01, -7.37057911e-01, -7.81796733e-01, ...,
        -1.16061272e-02, -1.01853314e-02,  0.00000000e+00],
       [-1.47015223e+00, -8.13452799e-01, -7.82557107e-01, ...,
        -1.16061272e-02, -1.01853314e-02,  0.00000000e+00],
       [-1.67577223e-03,  2.06788350e+00, -1.12510681e-01, ...,
        -1.16061272e-02, -1.01853314e-02,  0.00000000e+00]])

In [13]:
#labels for training
labels=labels_full.values
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [14]:
#Validation Technique
x_train, x_test, y_train, y_test = train_test_split(train_data_st, labels, test_size=0.2)
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.125)

print(x_train.shape)
print(x_test.shape)
print(x_validate.shape)

(712808, 115)
(203660, 115)
(101830, 115)


In [15]:
x_train_cnn = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
x_test_cnn = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
x_validate_cnn = np.reshape(x_validate, (x_validate.shape[0], x_validate.shape[1],1))
print(x_train_cnn.shape)
print(x_test_cnn.shape)
print(x_validate_cnn.shape)

(712808, 115, 1)
(203660, 115, 1)
(101830, 115, 1)


In [17]:


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model

from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from IPython.core.display import Image



# Build Model CNN_LSTM
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, strides=1, padding='same', input_shape = (train_data_st.shape[1], 1)))
model.add(Conv1D(filters=32, kernel_size=5, strides=1, padding='same'))
model.add(LSTM(32, activation = 'relu', return_sequences=True))
model.add(LSTM(16, return_sequences=True))  # returns a sequence of vectors of dimension 16
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(labels.shape[1],activation='softmax'))

modelName = 'CNN+LSTM'
keras.utils.plot_model(model, './'+modelName+'_Archi.png',show_shapes=True)
model.summary()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 115, 64)           384       
                                                                 
 conv1d_3 (Conv1D)           (None, 115, 32)           10272     
                                                                 
 lstm_2 (LSTM)               (None, 115, 32)           8320      
                                                                 
 lstm_3 (LSTM)               (None, 115, 16)           3136      
                                                                 
 flatten_1 (Flatten)         (None, 1840)              0         
                                                                 
 dense_3 (Dense)        

In [18]:
# Build Model
inp = Input(shape=(train_data_st.shape[1], 1))
C = Conv1D(filters=32, kernel_size=5, strides=1)(inp)

C11 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(C)
A11 = Activation("relu")(C11)
C12 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A11)
S11 = Add()([C12, C])
A12 = Activation("relu")(S11)
M11 = MaxPooling1D(pool_size=5, strides=2)(A12)


C21 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M11)
A21 = Activation("relu")(C21)
C22 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A21)
S21 = Add()([C22, M11])
A22 = Activation("relu")(S11)
M21 = MaxPooling1D(pool_size=5, strides=2)(A22)


C31 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M21)
A31 = Activation("relu")(C31)
C32 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A31)
S31 = Add()([C32, M21])
A32 = Activation("relu")(S31)
M31 = MaxPooling1D(pool_size=5, strides=2)(A32)


C41 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M31)
A41 = Activation("relu")(C41)
C42 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A41)
S41 = Add()([C42, M31])
A42 = Activation("relu")(S41)
M41 = MaxPooling1D(pool_size=5, strides=2)(A42)


C51 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M41)
A51 = Activation("relu")(C51)
C52 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A51)
S51 = Add()([C52, M41])
A52 = Activation("relu")(S51)
M51 = MaxPooling1D(pool_size=5, strides=2)(A52)

F1 = Flatten()(M51)

D1 = Dense(32)(F1)
A6 = Activation("relu")(D1)
D2 = Dense(32)(A6)
D3 = Dense(labels.shape[1])(D2)
A7 = Activation("softmax")(D3)

model = Model(inputs=inp, outputs=A7)


keras.utils.plot_model(model, './Deep_residual_CNN_model.png', show_shapes=True)

modelName='Deep residual CNN'

model.summary()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 115, 1)]     0           []                               
                                                                                                  
 conv1d_4 (Conv1D)              (None, 111, 32)      192         ['input_1[0][0]']                
                                                                                                  
 conv1d_5 (Conv1D)              (None, 111, 32)      5152        ['conv1d_4[0][0]']               
                                                                                                  
 activation (Activation)       

In [20]:
'''
For training the networks, we used Adam optimization method with the learning rate, beta-1, 
and beta-2 of 0.001, 0.9, and 0.999, respectively. 
Learning rate is decayed exponentially with the decay factor of 0.75 every 10000 iterations.
'''
adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True) 

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            lr=0.00001)
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 10,
                          verbose = 1,
                          restore_best_weights = True)

checkpoint = ModelCheckpoint('./'+modelName+'.h5',
                            monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=1)

In [ ]:
epochs = 15
batch_size = 512
history = model.fit(x_train_cnn,y_train, batch_size=batch_size,
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_validate_cnn,y_validate),
    #validation_split=0.10,
    callbacks=[learning_rate_reduction, checkpoint] 
)

#1. Function to plot model's validation loss and validation accuracy
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1), model_history.history['accuracy'], '--*', color = (1,0,0))
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1), model_history.history['val_accuracy'], '-^', color = (0.7,0,0.7))
    axs[0].set_title('Model '+modelName+' Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[0].grid('on')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1), model_history.history['loss'],'-x', color = (0,0.5,0))
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1), model_history.history['val_loss'],  '-.D', color = (0,0,0.5))
    axs[1].set_title('Model '+modelName+' Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    axs[1].grid('on')
    plt.savefig('./'+modelName+'.jpg',dpi=600, quality = 100, optimize = True)
    plt.show()

plot_model_history(history)
with open('./History_'+modelName, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

Epoch 1/15
 724/1392 [==============>...............] - ETA: 1:26 - loss: 0.4088 - accuracy: 0.8066